# Project: LSTM_module (Server)

학습된 모델로 음성파일 스팸 예측 (앱과 소켓 통신)

## 순서

1. import: 필요한 모듈 import
2. 파일 불러오기: 모델 학습에 사용된 객체, 변수 등 불러오기
3. 오디오 파일 형식 변환: m4a -> flac 형태로 변환
4. STT: 음성파일 STT변환
5. 전처리: 텍스트 데이터 분리 및 토큰화
6. 예측: 스팸 탐지
7. 전체 동작 (스레드)
8. 소켓 연결

## 모듈 import

In [1]:
import os
import sys
import json
import pickle
import re
import numpy as np
import time

import speech_recognition as sr
from pydub import AudioSegment

import tensorflow as tf
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import urllib.request

import socket
import threading

## GPU 셋팅

GPU 사용 시 활성화

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

## 파일 이름 선언

필요 변수, 모델, 토큰 객체 등 파일 이름 선언

In [3]:
file_name = "LSTM_module_ver4.0"
data_path = "CollectedData_y-call/"

## 파일 불러오기

tokenizer 객체 불러오기

In [4]:
with open(file_name+'_tokenizer.pickle', 'rb') as f:
    tokenizer = pickle.load(f)

변수 불러오기

In [5]:
with open(file_name+'_variable.json') as f:
    var = json.load(f)

불용어 단어 불러오기

In [6]:
with open(file_name+'_stopwords.json') as f:
    stopwords = json.load(f)

불러온 변수 초기화

In [7]:
min_data = var['min_data'] # 데이터 문자열 최소 길이
data_length = var['data_length'] # 데이터 문자열 최대 길이
split_list = var['split_list'] # 문장 단위로 분리하기 위한 글자 리스트
max_len = var['max_len'] # 전체 데이터 셋 길이 설정 (메일의 최대 길이)
trunc_type = var['trunc_type']
padding_type = var['padding_type']

print('min_data:',min_data)
print('data_length:',data_length)
print('max_len:',max_len)
print('trunc_type:',trunc_type)
print('padding_type:',padding_type)

min_data: 10
data_length: 100
max_len: 100
trunc_type: post
padding_type: post


## 오디오 파일 형식 변환

**formatTransform**: 오디오 파일 형식 변환 함수

In [8]:
def formatTransform(file_name):
    recording = AudioSegment.from_file(data_path+file_name)
    recording = recording + 6 # 볼륨 향상
    formatFlac = file_name.replace(".m4a",".flac")
    recording.export(data_path+formatFlac, format='flac', parameters=["-q:a", "10", "-ac", "1"])
    print("Convert : " + file_name + " -> " + formatFlac)
    file_size = os.path.getsize(data_path+formatFlac)
    os.remove(data_path+file_name) # 변환 전 파일 제거
    print("remove : " + file_name)
    
    return formatFlac

## STT 변환

**STTtransform**: 음성파일 STT변환함수
* `language`: 언어 설정
* 약 80~100MB 용량 제한
* 인식된 음성이 없거나 용량이 큰 경우 오류

In [9]:
def STTtransform(file_name):
    r = sr.Recognizer()
    harvard = sr.AudioFile(data_path+file_name) # 80MB 용량 제한
    with harvard as source:
        audio = r.record(source)
        
    try:
        t = r.recognize_ibm(audio, language='ko-KR') # 구글로 바꿔두기
    except:
        t = ""
        print("음성인식이 불가하거나 용량이 너무 큽니다.")
#     os.remove(file_name) # 음성파일 제거
#     print("remove : " + file_name)
    
    return t

## 데이터 전처리

Okt 객체 선언

In [10]:
okt = Okt()

### 1. 텍스트 데이터 분리

**findIndex**: 문장이 끝나는 부분의 index값을 찾기위한 함수

In [11]:
def findIndex(text, split_list, start_index):
    index_list = []
    for i in split_list:
        index = text.find(i, start_index)
        index_list.append(index)
        
    index_list = [i for i in index_list if i not in [-1]]
    if index_list == []:
        return -1
    
    index = min(index_list)
    
    return index

### 2. 토큰화 및 불용어 처리
**preTreatment**: 텍스트 데이터 분리 및 토큰화, 불용어 처리 함수

In [12]:
def preTreatment(text):
    global stopwords
    
    text = re.sub("네|[^가-힣 ]", "", text) # 특수문자 제거
    index = findIndex(text, split_list, data_length)

    i = 0
    t_list = []
    while index != -1:
        x = text[i:index+2]
        t_list.append(x)

        i = index+2
        index = findIndex(text, split_list, i+data_length)
    else:
        x = text[i:]
        if len(x) > min_data:
            t_list.append(x) # 텍스트 마지막 부분 추가
               
    token_list = []
    cnt = 1
    for t in t_list:
        #print('{}. {}'.format(cnt, t))
        print(str(cnt)+'. (통화 내용을 확인하려면 주석을 해제해주세요.)')
        cnt += 1
        
        temp = okt.morphs(t, stem=True) # 토큰화
        temp = [word for word in temp if not word in stopwords] # 불용어 처리
        token_list.append(temp)
        
    return token_list

## 모델 예측

학습된 모델 불러오기

In [13]:
model = tf.keras.models.load_model(file_name+'.h5')
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 100)         657100    
_________________________________________________________________
lstm_6 (LSTM)                (None, None, 128)         117248    
_________________________________________________________________
lstm_7 (LSTM)                (None, None, 128)         131584    
_________________________________________________________________
lstm_8 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 129       
Total params: 1,037,645
Trainable params: 1,037,645
Non-trainable params: 0
_________________________________________________________________


**sentiment_predict**: 정수 인코딩 후 모델 예측 함수

In [14]:
def sentiment_predict(sentence_list):
    
    score_sum = 0.0
    cnt = 1
    for s in sentence_list:    
        data = tokenizer.texts_to_sequences([s]) # 단어를 숫자값, 인덱스로 변환하여 저장
        pad_new = pad_sequences(data, maxlen = max_len) # 빈 부분은 0으로 패딩
        score = float(model.predict(pad_new))
        
        print("{}. [{:.2f}% 확률로 스팸입니다.]".format(cnt, score * 100))
        cnt += 1
        score_sum += score    
    score_result = score_sum / len(sentence_list)
    
    return score_result

## 다중 클라이언트 소켓 통신을 위한 스레드 처리

데이터 수신 및 전송

In [15]:
def threaded(conn, addr, group):
    print('Connect by',addr)
    while True:
        try:
            data = conn.recv(1024) # 파일명, 사이즈 수신
            if not data: # 소켓 연결 끊기면 연결 해제
                print('Disconnected by',addr)
                group.remove(conn)
                break
                
            file_info = data.decode()
            file_name, file_size = file_info.split("/")
            file_name = file_name.replace('.voicecall','.m4a')
            print('Receive File Path:',file_name)
            print('Receive File Size:',file_size)

            data = conn.recv(1024) # 파일 수신
            data_transferred = len(data)
            with open("CollectedData_y-call/"+file_name, "wb") as f:
                try:
                    while data:
                        f.write(data)
                        data = conn.recv(1024)
                        data_transferred += len(data)
                        
                        if data_transferred == int(file_size): # 파일 다 받으면 break
                            f.write(data)
                            break
                except Exception as ex:
                    print(ex)

            print("File is saved [byte:"+str(data_transferred)+"]")
            
            audio = formatTransform(file_name) # 오디오 파일 형식 변환
            text = STTtransform(audio) # STT 변환
            data = preTreatment(text) # 전처리
            
            if data != []:
                score = sentiment_predict(data) # 예측
                txt = "{:.2f}\n".format(score * 100) # Client측 readLine 함수로 인한 개행문자(\n)필요
                print('{:.2f}% 확률로 스팸입니다.'.format(score * 100))
                if score > 0.5:
                    print('<스팸 메시지>')
                    print(text)
                    
            else:
                txt = "-1\n"
                print('녹음길이가 너무 짧거나 파일의 용량이 너무 큽니다.')
            
            sendData = txt
            for c in group:
                if c is conn:
                    c.sendall(bytes(sendData,'UTF-8')) # 수신된 파일을 보낸 Client에게만 전송
                    print('Send Data : '+txt,end="")
                    print('-------------------------------------------------------------------')
        except:
            # 클라이언트 소켓 강제 종료 시 (ex : 네트워크 변경)
            print('예외발생')
            print('Disconnected by',addr)
            group.remove(conn)
            break
    conn.close()

## 소켓 연결

IP, PORT 설정

In [16]:
host = ''
port = 50000

소켓 생성 후 연결 대기

In [ ]:
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
server_socket.bind((host, port))
server_socket.listen()
print("Listening")

group = []
while True:
    conn, addr = server_socket.accept()
    group.append(conn)
    thx = threading.Thread(target=threaded, args=(conn, addr, group))
    thx.start()
    
server_socket.close()
print('서버 종료')

Listening
Connect by ('121.182.164.108', 31778)
예외발생
Disconnected by ('121.182.164.108', 31778)
Connect by ('121.182.164.108', 31828)
예외발생
Disconnected by ('121.182.164.108', 31828)
Connect by ('121.182.164.108', 14220)
Disconnected by ('121.182.164.108', 14220)
Connect by ('121.182.164.108', 14228)
Disconnected by ('121.182.164.108', 14228)
Connect by ('121.182.164.108', 14240)
Disconnected by ('121.182.164.108', 14240)
Connect by ('121.182.164.108', 14248)
Disconnected by ('121.182.164.108', 14248)
Connect by ('121.182.164.108', 31880)
예외발생
Disconnected by ('121.182.164.108', 31880)
Connect by ('121.182.164.108', 31916)
예외발생
Disconnected by ('121.182.164.108', 31916)
Connect by ('121.182.164.108', 32072)
Receive File Path: [] 통화 녹음 01092713205_211014_110612.m4a
Receive File Size: 1117955
File is saved [byte:1117955]
Convert : [] 통화 녹음 01092713205_211014_110612.m4a -> [] 통화 녹음 01092713205_211014_110612.flac
remove : [] 통화 녹음 01092713205_211014_110612.m4a
음성인식이 불가하거나 용량이 너무 큽니다.
녹음길이가 

Disconnected by ('121.182.164.108', 32224)
Connect by ('121.182.164.108', 32236)
Receive File Path: [] 통화 녹음 01092713205_211014_102133.m4a
Receive File Size: 1502487
File is saved [byte:1502487]
Convert : [] 통화 녹음 01092713205_211014_102133.m4a -> [] 통화 녹음 01092713205_211014_102133.flac
remove : [] 통화 녹음 01092713205_211014_102133.m4a
음성인식이 불가하거나 용량이 너무 큽니다.
녹음길이가 너무 짧거나 파일의 용량이 너무 큽니다.
Send Data : -1
-------------------------------------------------------------------
Disconnected by ('121.182.164.108', 32236)
Connect by ('121.182.164.108', 32274)
Receive File Path: [] 통화 녹음 01092713205_211014_100630.m4a
Receive File Size: 124331
File is saved [byte:124331]
Convert : [] 통화 녹음 01092713205_211014_100630.m4a -> [] 통화 녹음 01092713205_211014_100630.flac
remove : [] 통화 녹음 01092713205_211014_100630.m4a
음성인식이 불가하거나 용량이 너무 큽니다.
녹음길이가 너무 짧거나 파일의 용량이 너무 큽니다.
Send Data : -1
-------------------------------------------------------------------
Receive File Path: [] 통화 녹음 01092713205_211014_100510.m4a
Re

Convert : [] 통화 녹음 01092713205_211014_100322.m4a -> [] 통화 녹음 01092713205_211014_100322.flac
remove : [] 통화 녹음 01092713205_211014_100322.m4a
음성인식이 불가하거나 용량이 너무 큽니다.
녹음길이가 너무 짧거나 파일의 용량이 너무 큽니다.
Send Data : -1
-------------------------------------------------------------------
Receive File Path: [] 통화 녹음 01092713205_211014_095922.m4a
Receive File Size: 704884
File is saved [byte:704884]
Convert : [] 통화 녹음 01092713205_211014_095922.m4a -> [] 통화 녹음 01092713205_211014_095922.flac
remove : [] 통화 녹음 01092713205_211014_095922.m4a
음성인식이 불가하거나 용량이 너무 큽니다.
녹음길이가 너무 짧거나 파일의 용량이 너무 큽니다.
Send Data : -1
-------------------------------------------------------------------
Receive File Path: [] 통화 녹음 01092713205_211014_095847.m4a
Receive File Size: 268155
File is saved [byte:268155]
Convert : [] 통화 녹음 01092713205_211014_095847.m4a -> [] 통화 녹음 01092713205_211014_095847.flac
remove : [] 통화 녹음 01092713205_211014_095847.m4a
음성인식이 불가하거나 용량이 너무 큽니다.
녹음길이가 너무 짧거나 파일의 용량이 너무 큽니다.
Send Data : -1
----------------